In [ ]:
__author__ = 'Nick Sarris (ngs5st)'

import os
import time
import random
import numpy as np
import pandas as pd
import lightgbm as lgb

from datetime import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from scipy.stats import skew, boxcox
from sklearn.preprocessing import StandardScaler
import itertools

print(os.listdir("./data"))

In [ ]:
def seed_everything(seed=1235):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(1235)

In [ ]:
start_time = time.time()
print("Loading Data ...")

directory = "./data/"

order_prior = pd.read_csv(directory + 'order_products__prior.csv', dtype={
    'order_id': np.uint32, 'product_id': np.uint16, 'add_to_cart_order':np.uint8,
    'reordered': bool})
orders = pd.read_csv(directory + 'orders.csv', dtype={
    'order_id':np.uint32, 'user_id': np.uint32, 'eval_set': 'category',
    'order_number':np.uint8, 'order_dow': np.uint8, 'order_hour_of_day': np.uint8})

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
print("Establish Labels ...")

orders = orders.loc[orders.eval_set == 'prior', :]
orders_user = orders[['order_id', 'user_id']]
labels = pd.merge(order_prior, orders_user, on='order_id')
labels = labels.loc[:, ['user_id', 'product_id']].drop_duplicates()
labels.to_pickle('./data/previous_products.pkl')

print("--- %s seconds ---" % (time.time() - start_time))